# Scrapping Twitter Tweet

## Using Tweepy Library

In [1]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
import string
import preprocessor as p
import os
import time

In [2]:
# Twitter credentials
# Obtain them from your twitter developer accountconsumer_key = <your_consumer_key>
# Authenticate to Twitter
auth = tweepy.OAuthHandler("gb2sL1bqzj5Gj6kLppBvPA", "t9W01e2PU4hOlYIxXEAIy83pCKPi7SX96Q9xgJr4")
auth.set_access_token("780290756-CC7evf7VBvrOCrS1uCX5V9Oj8YRUKfpo0AUdEfBK","0X7brWyLUJAJxeBaD4NJqq9C4OH0DXtz4oYw2Q93BWRmv")
api = tweepy.API(auth)# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


## Search by Hashtag

In [3]:
def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="id", since=date_since, tweet_mode='extended').items(numTweets)# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet# Begin scraping the tweets individually:
        noTweets = 0
        for tweet in tweet_list:# Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text# Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet# increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')# Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_#blacklivesmatter_tweets.csv'# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [4]:
# Initialise these variables:
search_words = "#TEDxJakarta OR #tedxjakarta"
date_since = "2014-03-01" #the event
numTweets = 200
numRuns = 1# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 0
time take for 1 run to complete is 0.02 mins
Scraping has completed!
Total time taken to scrap is 15.35 minutes.


## Search by Word

In [18]:
tweets = []

def text_query_to_csv(text_query,count):
    try:
    # Pulling individual tweets from query
        for tweet in api.search(q=text_query, count=count):

          # Adding to list that contains all tweets
          tweets.append((tweet.created_at,tweet.id,tweet.text))

          # Creation of dataframe from tweets list
          tweetsdf = pd.DataFrame(tweets,columns=['Datetime', 'Tweet Id', 'Text'])

          # Converting dataframe to CSV
          # Obtain timestamp in a readable formatto_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')# Define working path and filename
            path = os.getcwd()
            filename = path + '/data/' + to_csv_timestamp + '_#TEDxJKT2_tweets.csv'# Store dataframe in csv with creation date timestamp
            db_tweets.to_csv(filename, index = False)
            
            program_end = time.time()
            print('Scraping has completed!')
            print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)

In [20]:
# Input search query to scrape tweets and name csv file
# Max recent tweets pulls x amount of most recent tweets from that user
text_query = 'TEDxJakarta'
count = 500

# Calling function to query X amount of relevant tweets and create a CSV file
text_query_to_csv(text_query, count)

## Refference
https://medium.com/@leowgriffin/scraping-tweets-with-tweepy-python-59413046e788

https://towardsdatascience.com/how-to-access-twitters-api-using-tweepy-5a13a206683b